# Importing dependencies

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
# Loading pre-trained VGG19 architecture
model = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
model.trainable = False

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 36s 0us/step


<img src='https://miro.medium.com/v2/resize:fit:720/format:webp/1*1ztswIVGxwwTYPsTEygSJg.png'>

In [9]:
# defining content and style layers

content_layers = ['block5_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

In [10]:
# Create a model that outputs the content and style representations of the input image
content_model = tf.keras.Model(inputs=model.input, outputs=[model.get_layer(layer).output for layer in content_layers])
style_model = tf.keras.Model(inputs=model.input, outputs=[model.get_layer(layer).output for layer in style_layers])

<h3>content_loss= ∑(Ccontent-Tcontent)²/2</h3>

Ccontent = features extracted through the network of Original Content image.
and Tcontent = features of the content of Target image.

We will need to update out Tcontent such that the Content loss is minimum. We will be using the second convolutional layer in 4th stack of convolutional layer(conv4_2) to compare the content and target image.

In [33]:
def get_content_loss(base,target):
    return tf.reduce_mean(tf.square(base-target))

<h4>Gram Matrix gives the correlation matrix between the features/filters.<br><br>
    <img src='https://miro.medium.com/v2/resize:fit:828/format:webp/1*mbeEuJLyw_2atq3pi1PgAA.png'>

In [34]:
def gram_matrix(input_tensor):
    feature_maps=int(input_tensor.shape[-1])
    a=tf.reshape(input_tensor.shape[-1])
    n=tf.shape(a)[0]
    gram=tf.matmul(a,a,transpose_a=True)
    return gram/tf.cast(n,tf.float32)

<h2>Style Loss:</h2>

We want our target images style to be similar to our style image. So we define a metric to know how close is the style of target to the original style.

We compare the gram matrix of the target and style image in every convolutional layer, to see how close is the style.

<h3>Lstyle = ∑w(Cstyle-Tstyle)²</h3>

Cstyle = gram matrix of original image style of each layer.<br>
Tstyle = gram matrix of the target image.<br>
w = style weight at each layer<br>

In [35]:
def style_loss(base,gram_target):
    return tf.reduce_mean(tf.square(gram_matrix(base)-gram_target))

<h2> Total Loss:</h2>

We cannot add the content loss and style loss directly and use that as total loss. We need to apply the style over our content. Even from this statement we can understand that the ratio od them can not be equal. i mean we cannot give equal weights to them. We want to neural network to adjust some weight based on what target we want. So we take the weight as a linear sum with parameters α and β.

<h3>Ltotal = αLcontent + βLstyle</h3><br>
Thats it!!, this is all math we need for style transfer. We need to reduce the total loss and adjust Tstyle & T content to get a target image which has content of the content image and style of out original style image.


In [36]:
def compute_loss(model,base,style_reference_image,content_weight,style_weight):
    content_output=model(base)
    style_output=model(style_reference_image)
    
    content_loss=0
    style_loss=0
    
    for target_content, comb_content in zip(content_output, content_model(base)):
        content_loss += get_content_loss(comb_content[0], target_content)
    
    for target_style, comb_style in zip(style_outputs, style_model(base_image)):
        style_loss += get_style_loss(comb_style[0], target_style)
        
    content_loss=content_loss*content_weight/len(content_layers)
    style_loss=style_loss*style_weight/len(style_layers)
    
    return total_loss,content_loss,style_loss


In [37]:
def compute_gradients(config, base_image, style_reference_image, content_weight, style_weight):
    # Open a gradient tape to record operations for automatic differentiation
    with tf.GradientTape() as tape:
        # Compute the total loss, content loss, and style loss using the provided model
        loss, content_loss, style_loss = compute_loss(model, base_image, style_reference_image, content_weight, style_weight)

    # Calculate the gradients of the total loss with respect to the base image
    gradients = tape.gradient(loss, base_image)

    # Return the computed gradients along with the individual losses
    return gradients, loss, content_loss, style_loss


In [38]:
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.vgg19 import preprocess_input

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    # Load the image from the file path
    img = keras_image.load_img(image_path, target_size=target_size)
    
    # Convert the PIL image to a NumPy array
    img_array = keras_image.img_to_array(img)
    
    # Expand the dimensions to create a batch of size 1
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess the image using the VGG19 preprocess_input function
    img_preprocessed = preprocess_input(img_array)
    
    # Convert the NumPy array back to a TensorFlow tensor
    img_tensor = tf.convert_to_tensor(img_preprocessed)
    
    return img_tensor

In [39]:
def run_style_transfer(content_path, style_path, num_iterations=1000, content_weight=1e3, style_weight=1e-2):
    content_image = load_and_preprocess_image(content_path)
    style_image = load_and_preprocess_image(style_path)

    # Initialize the generated image with the content image
    generated_image = tf.Variable(content_image, dtype=tf.float32)

    # Adam optimizer
    optimizer = tf.optimizers.Adam(learning_rate=0.5, beta_1=0.99, epsilon=1e-1)

    # Create a checkpoint to save the generated image during training
    checkpoint = tf.train.Checkpoint(generated_image=generated_image)

    # Create a custom configuration to use during training
    config = {
        'num_iterations': num_iterations,
        'content_weight': content_weight,
        'style_weight': style_weight
    }

    for i in range(num_iterations):
        gradients, loss, content_loss, style_loss = compute_gradients(config, generated_image, style_image, content_weight, style_weight)
        optimizer.apply_gradients([(gradients, generated_image)])
        clipped_image = tf.clip_by_value(generated_image, 0.0, 255.0)
        generated_image.assign(clipped_image)

        if i % 100 == 0:
            print(f"Iteration {i}: Total Loss: {loss.numpy()}, Content Loss: {content_loss.numpy()}, Style Loss: {style_loss.numpy()}")

        if i % 500 == 0:
            # Save the generated image
            checkpoint.save(file_prefix='./checkpoints/ckpt-')

    return generated_image.numpy()

In [40]:
content_path = 'profile.jpg'
style_path = 'monet.jpeg'
output_image = run_style_transfer(content_path, style_path)

# Display the result
plt.imshow(output_image/255.0)
plt.show()

InvalidArgumentError: {{function_node __wrapped__Sub_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [14,512] vs. [7,7,512] [Op:Sub] name: 